# Answering Business Questions using SQL. Music Store project

The goal for this project is help the decision makers of the store deal with their daily business questions.  

# Add new artists to the store
The Chinook record store has signed a deal with a new record label. Select the first three albums that will be added to the store out of these 4:
 - Regal/Hip-hop
 - Red Tone/Punk
 - Meteor and the Girls/Pop
 - Slim Jim Bites/Blues
The record label wants their albums to be advertised in the USA, so we will need to write a query to find out which genre sell the most tracks in the USA.

# Sales support agents' performance
Each custumer gets assigned to a sales support agent after making their first purchase. 

Analyze the purchases of customers belonging to each employee to see if one is performing better than the other.


# Analyze the sales data by country
Calculate data for each country on the total number of custumers, total value of sales, average value of sales per customer and average order value.

Use the country value from the customer table and ignore the country from the billing address in the invoice table.

Countries with only one custumer, should be grouped as "Other".

# Should the store change their purchasing strategy?
Currently, the store allows customers to either purchase a whole album or pucrhase a collection of one or more individual tracks.

The management is considering to change that strategy to save money, and purchase only the most popular tracks from each album from record companies instead of every track.

We need to find out what percentage of purchases are individual tracks vs whole albums, so they have a better understanding of the effect that this decision would have on the overall revenue.

We'll be working with a modified version of a database called Chinook. The Chinook database contains information about a fictional digital music shop - a mini-iTunes store.

The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables:
- employee
- customer
- invoice
- invoice_line
- playlist
- playlist_track
- track
- media_type
- genre
- artist
- album

The database schema is included in the folder under the name chinook-schema.svg.

You can check more about our database here:
https://github.com/lerocha/chinook-database

Connect Jupyter Notebook to the database file

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

Getting familiar with our data

In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table", "view")

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [4]:
%%sql
SELECT * FROM media_type
LIMIT 5

Done.


media_type_id,name
1,MPEG audio file
2,Protected AAC audio file
3,Protected MPEG-4 video file
4,Purchased AAC audio file
5,AAC audio file


In [5]:
%%sql
SELECT * FROM customer
LIMIT 5

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


# Add new artists to the store
Write a query that returns each genre, with the number of tracks sold in the USA, in absolute numbers and in percentages.

In [36]:
%%sql

WITH USA_customers AS
   (
     SELECT 
         i.*,
         il.*
      FROM invoice i
      INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
      WHERE i.billing_country = "USA"
   )
SELECT
  g.name Genre,
  SUM(USA_customers.quantity) Total_Sales,
  ROUND(CAST(SUM(USA_customers.quantity) AS FLOAT) / (
                                                 SELECT COUNT(USA_customers.invoice_line_id) FROM USA_customers
                                                ),3) Percentage_Sold
FROM track t
INNER JOIN USA_customers ON USA_customers.track_id = t.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
  

Done.


Genre,Total_Sales,Percentage_Sold
Rock,561,0.534
Alternative & Punk,130,0.124
Metal,124,0.118
R&B/Soul,53,0.05
Blues,36,0.034
Alternative,35,0.033
Latin,22,0.021
Pop,22,0.021
Hip Hop/Rap,20,0.019
Jazz,14,0.013


The best selling genres in the USA out of the 4, are Punk, Blues and Pop. So the advise would be to go with the artists that represent these genres. On the other hand, after making this analysis, we can see that 53% of the tracks/albums sold are Rock. We should definitely take that into consideration when we purchase futuer albums.

# Sales support agents' performance
Write a query that finds the total dollar amount of sales assigned to each sales support agent within the company.


In [41]:
%%sql

SELECT
   e.first_name || " " || e.last_name Employee_Name,
   e.hire_date Hire_Date,
   SUM(i.total) Total_Sales
FROM employee e
INNER JOIN customer c ON c.support_rep_id = e.employee_id
INNER JOIN invoice i ON i.customer_id = c.customer_id
WHERE e.title = "Sales Support Agent"
GROUP BY 1
ORDER BY 3 DESC

Done.


Employee_Name,Hire_Date,Total_Sales
Jane Peacock,2017-04-01 00:00:00,1731.5100000000039
Margaret Park,2017-05-03 00:00:00,1584.0000000000034
Steve Johnson,2017-10-17 00:00:00,1393.920000000002


By looking at this analysis, we may say that the sales performance is directly related to the employee's hiring date. Either because they have more time to sell or have more time and training invested in them. We will need to investigate that forward.

# Analyze the sales data by country
Write a query that collects data on purchases from different countries:
- Where a country has only one customer, collect them into an "Other" group
- The result should be sorted by the total sales from highest to lowest with the "Other" group at the very bottom
- For each country include: total number of customers, total value of sales, average value of sales per customer, average order value

In [46]:
%%sql
WITH country_other AS
    (
     SELECT
        CASE
            WHEN
                (SELECT COUNT(*)
                   FROM customer
                  WHERE country = c.country
                 ) = 1 THEN "OTHER"
            ELSE c.country
        END AS country,
        c.customer_id,
        i.*
      FROM invoice i
      INNER JOIN customer c ON c.customer_id = i.customer_id
    )
SELECT
 Country,
 Customers,
 Total_Sales,
 Average_per_Customer,
 Average_Order_Value
FROM
 (
  SELECT
   Country,
   COUNT(DISTINCT customer_id) Customers,
   ROUND(SUM(total), 3) Total_Sales,
   ROUND(SUM(total) / COUNT(DISTINCT customer_id), 3) Average_per_Customer,
   ROUND(SUM(total) / COUNT(DISTINCT invoice_id), 3) Average_Order_Value,
   CASE 
       WHEN country = "OTHER" THEN 1
                              ELSE 0
   END AS sort
FROM country_other
GROUP BY 1
ORDER BY sort ASC, total_sales DESC)


Done.


Country,Customers,Total_Sales,Average_per_Customer,Average_Order_Value
USA,13,1040.49,80.038,7.943
Canada,8,535.59,66.949,7.047
Brazil,5,427.68,85.536,7.011
France,5,389.07,77.814,7.781
Germany,4,334.62,83.655,8.161
Czech Republic,2,273.24,136.62,9.108
United Kingdom,3,245.52,81.84,8.769
Portugal,2,185.13,92.565,6.384
India,2,183.15,91.575,8.721
OTHER,15,1094.94,72.996,7.449


Most customers are from the USA and Canada, that makes total sense as our store's focus is on USA customers. 

What's interesting to see is that Czech Republic, Portugal and India custumers have the highest average in our list, so we might explore more that opportunity. We will definitely need to have more data in order to make that decision, as now we only have 3 customers in those markets.

# Should the store change their purchasing strategy?
Write a query that categorizes each invoice as either an album purchase or not and calculate the number of invoices and the purcentage of invoices.

We can ignore albums that have only one or two tracks (this doesn't concern maximizing revenue) or the case when custumers may decide to manually select every track from an album, then add a few individual tracks from other albums (this rarely happens). 

In [49]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
        il.invoice_id invoice_id,
        MIN(il.track_id) first_track_id
      FROM invoice_line il
     GROUP BY 1
    )
SELECT
   Album_Purchase,
   COUNT(invoice_id) Number_of_Invoices,
   CAST(count(invoice_id) AS FLOAT) / (
                                       SELECT COUNT(*) 
                                         FROM invoice) Percentage_of_Invoices
FROM
   (
    SELECT
      ift.*,
      CASE 
          WHEN
              (
               SELECT t.track_id
                 FROM track t
                WHERE t.album_id = (
                                    SELECT t2.album_id
                                      FROM track t2
                                     WHERE t2.track_id = ift.first_track_id
                                    )
               EXCEPT
              
               SELECT il2.track_id
                 FROM invoice_line il2
                WHERE il2.invoice_id = ift.invoice_id
              ) IS NULL
          AND
             (
              SELECT il2.track_id
                FROM invoice_line il2
               WHERE il2.invoice_id = ift.invoice_id
                 
              EXCEPT
                 
              SELECT t.track_id
                 FROM track t
                WHERE t.album_id = (
                                    SELECT t2.album_id
                                      FROM track t2
                                     WHERE t2.track_id = ift.first_track_id
                                    )
             ) IS NULL
          THEN "YES"
          ELSE "NO"
       END AS Album_Purchase
    FROM invoice_first_track ift
   )
GROUP BY Album_purchase

Done.


Album_Purchase,Number_of_Invoices,Percentage_of_Invoices
NO,500,0.8143322475570033
YES,114,0.18566775244299674


18% of the purchases are album purchases. Changing the sales strategy to only selling tracks, would mean losing this revenue, so we would advise against it.

Conclusions

As stated from the begining, the purpose of this guided project was to try and answer some business questions using SQL.
By looking at the data, we can conclude that to maximize their revenew our store should:
- have a focus on getting more albums and tracks from Rock artists
- invest in retaining and training their sales force
- explore other markets that might be more profitable outside of the US
- not change their strategy and continue allowing their custumers to buy albums, not only tracks